In [1]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import warnings 
warnings.simplefilter('ignore')

from get_GloVe_emb_ML import *
from proj2_helpers import *
from ML_CNN import * 

In [2]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [133]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [3]:
PP_POS_PATH = './Results/pp_pos_otpl_nd.txt' 
PP_NEG_PATH = './Results/pp_neg_otpl_nd.txt' 
PP_TEST_PATH = './Results/pp_test_otpl.txt'
VECTORS_PATH = './GloVe-1.2/vectors.txt'
VOCABULARY_PATH = './GloVe-1.2/vocab.txt'

In [4]:
vocabulary = {} 
with open(VOCABULARY_PATH, "r") as f:
    for line in f:
        tokens = line.strip().split()
        vocabulary[tokens[0]] = [float(x) for x in tokens[1:]]
vocabulary = {k: v[0] for k, v in vocabulary.items()}
VOCAB_SIZE = len(vocabulary)

In [5]:
DIM_EMB = 50

In [6]:
testsize = 0.2

In [7]:
pos = open(PP_POS_PATH, "r").read().splitlines()
neg = open(PP_NEG_PATH, "r").read().splitlines()
test = open(PP_TEST_PATH, "r").read().splitlines()

In [8]:
vectors = get_vectors(VECTORS_PATH)

In [10]:
full_df, X, y = get_train_emb_CNN(pos, neg, vectors, DIM_EMB)
full_df.head()

> process pos and neg datas to get X and y to perform ML
> computing mean of word vectors
> computing mean of word vectors
> X and y informations:
X shape: (173209, 50)
y shape: (173209,)


,sentiment,twt_vec,twt
44546,1,"[-0.327424375, 0.20842725, -0.19167375, -0.054...",smile outside one thing smile inside another h...
20501,1,"[-0.17488874999999998, 0.238007, 0.15211524999...",absolutely sure trust gene
60652,1,"[0.12295224999999999, 0.27231737500000003, -0....",offline gon instagram come follow help usernam...
3207,1,"[-0.18450114285714284, 0.39531614285714284, -0...",like sweet excite face straight board bus
34112,1,"[-0.3030275714285714, 0.09224271428571426, -0....",plenty fish sea stay fuck away nemo


### Split + PCA 

In [126]:
X_df = full_df.copy()
X_df = X_df.drop(['sentiment'],axis=1)
y_df = full_df.copy()
y_df = y_df.drop(['twt_vec','twt'],axis=1)

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=testsize, random_state=1)
X_train = np.vstack(np.array(X_train_df['twt_vec'].values))
X_test = np.vstack(np.array(X_test_df['twt_vec'].values))

In [129]:
y_train = np.asarray([[0,1] if y == -1 else [1,0] for y in y_train_df['sentiment'].values])
y_test = np.asarray([[0,1] if y == -1 else [1,0] for y in y_test_df['sentiment'].values])

In [130]:
scaler = StandardScaler()
# fit on train set only
scaler.fit(X_train)
# apply transform to train and test
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print('> performing PCA')
# create instance of PCA
pca = PCA(.95)
# fit PCA on train set only
pca.fit(X_train)
# apply on train and test 
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

print('Train set: X shape: ', X_train.shape, 'y shape:', y_train.shape)
print('Test set: X shape: ', X_test.shape, 'y shape:', y_test.shape)

> performing PCA
Train set: X shape:  (138567, 40) y shape: (138567, 2)
Test set: X shape:  (34642, 40) y shape: (34642, 2)


### Get embeddings

In [119]:
train_embedding_weights = np.zeros((len(X_train)+1, EMB_DIM))
for index, word in enumerate(X_train_df['twt']):
    train_embedding_weights[index,:] = vectors[word] if word in vectors else np.random.rand(EMB_DIM)
print(train_embedding_weights.shape)

(138568, 50)


### Tokenize and Pad sequences

In [120]:
tokenizer = Tokenizer(num_words=len(vocabulary), lower=True, char_level=False)
tokenizer.fit_on_texts(X_train_df['twt'].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train_df['twt'].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 38017 unique tokens.


In [121]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [122]:
test_sequences = tokenizer.texts_to_sequences(X_test_df['twt'].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define CNN

In [123]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [124]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(X_train)+1, EMB_DIM, 
                len(list(label_names)))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 50)       6928400     input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 49, 200)      20200       embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 48, 200)      30200       embedding_5[0][0]                
____________________________________________________________________________________________

### Train CNN

In [135]:
num_epochs = 3
batch_size = 34

In [136]:
hist = model.fit(train_cnn_data, y_train, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 124710 samples, validate on 13857 samples
Epoch 1/3
124710/124710 [==============================] - 237s 2ms/step - loss: 0.6202 - acc: 0.6434 - val_loss: 0.5614 - val_acc: 0.7009
Epoch 2/3
124710/124710 [==============================] - 244s 2ms/step - loss: 0.5604 - acc: 0.7031 - val_loss: 0.5446 - val_acc: 0.7132
Epoch 3/3
124710/124710 [==============================] - 247s 2ms/step - loss: 0.5416 - acc: 0.7167 - val_loss: 0.5532 - val_acc: 0.7061


In [137]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

34642/34642 [==============================] - 17s 498us/step


In [138]:
labels = [1, 0]

In [139]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [146]:
y_test_df["cnn_label"] = [0 if sent == -1 else 1 for sent in y_test_df["sentiment"].values]

In [148]:
sum(y_test_df.cnn_label==prediction_labels)/len(prediction_labels)

0.7072339934183939

### Apply the model to the test data

In [152]:
test_df, test_x = get_test_emb(test, vectors, DIM_EMB)

> process test data to get X_test and perform ML
> computing mean of word vectors
Test shape (10000, 50)


In [153]:
test_df

,Tweet_submission_id,twt_vec
0,1,"[-0.16490428571428575, 0.07959978571428572, -0..."
1,2,"[-0.21977688888888888, 0.31933866666666666, -0..."
2,3,"[-0.14929240000000002, 0.5985406, -0.461216, 0..."
3,4,"[-0.29876600000000003, 0.3397075714285714, -0...."
4,5,"[-0.20306328571428572, 0.43171842857142856, -0..."
...,...,...
9995,9996,"[-0.22906875000000002, 0.37710675000000005, -0..."
9996,9997,"[0.21504649999999997, 0.487047, -0.2139565, 0...."
9997,9998,"[-0.0004636153846153871, 0.19289269230769232, ..."
9998,9999,"[-0.39865766666666674, 0.216822, 0.03363399999..."


In [ ]:
test_data = test_df.drop(columns=["Words_Vectors","Mean_Word_Vector"])

In [ ]:
RES_PATH = './Results/pp_test_otpl.txt'
test_set = open(RES_PATH, "r").read().splitlines()

In [157]:
test_sequences = tokenizer.texts_to_sequences(test_set)
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [158]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

10000/10000 [==============================] - 3s 285us/step


In [159]:
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [160]:
prediction_labels = [-1 if pred == 0 else 1 for pred in prediction_labels]

In [161]:
test_id = test_df['Tweet_submission_id'].to_numpy()

In [162]:
create_csv_submission(test_id,prediction_labels, "./Submissions/CNN_GloVe.csv")